In [7]:
#load libraries for data wrangling
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import os
import dask.dataframe as dd
import csv
import importlib
import html
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


#load function files
import Functions 
from Functions import * 

In [13]:
importlib.reload(Functions)

<module 'Functions' from '/home/livtollanes/10.jan-thesis/Code/Functions.py'>

### Data inspection
Loading data with dask

In [14]:
# data_path = "/home/livtollanes/SocialMarkers"
# data = load_data(data_path)

# #what does data contain?
# data.keys()

dict_keys(['marker_bios', 'marker_followers_bios', 'marker_followers', 'marker_friends', 'readme'])

In [15]:
# #inspect content of marker follower bios
# data['marker_followers_bios'].head()


ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+-----------------------+---------+----------+
| Column                | Found   | Expected |
+-----------------------+---------+----------+
| default_profile       | float64 | int64    |
| default_profile_image | float64 | int64    |
| followers             | float64 | int64    |
| friends               | float64 | int64    |
| id                    | float64 | int64    |
| likes                 | float64 | int64    |
| lists                 | float64 | int64    |
| protected             | float64 | int64    |
| timestamp_utc         | float64 | int64    |
| tweets                | float64 | int64    |
| verified              | float64 | int64    |
+-----------------------+---------+----------+

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'default_profile': 'float64',
       'default_profile_image': 'float64',
       'followers': 'float6

In [12]:
# subset = data['marker_followers_bios'][['twitter_id', 'id', 'screen_name', 'name', 'description', 'location']]
# #subset.head()


ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------+---------+----------+
| Column | Found   | Expected |
+--------+---------+----------+
| id     | float64 | int64    |
+--------+---------+----------+

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'id': 'float64'}

to the call to `read_csv`/`read_table`.

Alternatively, provide `assume_missing=True` to interpret
all unspecified integer columns as floats.

In [1]:
#what class of object is subset?
# len(subset.id)

NameError: name 'subset' is not defined

In [ ]:



# #extract marker_bios and marker_followers_bios from the dict as individual dataframes
# m_followers_bios = data['marker_followers_bios']

# #compute to pandas chunked dataframe
# ma_followers_bios_chunks = marker_followers_bios.to_delayed()
# first_chunk_marker_followers_bios = marker_followers_bios_chunks[0].compute()
# print(type(first_chunk_marker_followers_bios))



# total_rows_marker_followers_bios = sum(chunk.compute().shape[0] for chunk in marker_followers_bios_chunks)

# print(total_rows_marker_followers_bios)

# data['marker_followers_bios'].info()

# #inspect number of rows 
# data['marker_followers_bios'].shape[0].comopute()




# num_rows = data['marker_followers_bios'].shape[0].compute()

# print(num_rows)

# #Print the first few rows of each DataFrame
# for name, df in data.items():
#     print(f"First few rows of '{name}':")
#     if isinstance(df, str):
#         print(df)
#     else:
#         print(df.head())

dict_keys(['marker_bios', 'marker_followers_bios', 'marker_followers', 'marker_friends', 'readme'])

In [ ]:
# #Print the first 10 rows of the 'description' and 'location' in the 'marker_bios' and 'marker_followers_bios' DataFrames
# for name, df in data.items():   
#     if name == 'marker_bios' or name == 'marker_followers_bios':
#         print(f"First 10 rows of '{name}':")
#         print(df[['description','location']].head(10))  


# #print and compute the number of unique locations in marker_followers_bios
# print('Number of unique locations in marker_bios: {}'.format(len(data['marker_bios']['location'].unique())))
# #make a list of unique locations in marker_bios
# unique_locations = data['marker_bios']['location'].unique()


# print(unique_locations.compute())

# # count the number of unique locations in marker_followers_bios
# print('Number of unique locations in marker_followers_bios: {}'.format(len(data['marker_followers_bios']['location'].unique())))

# #inspect row number 287456 in marker_followers_bios
# print(data['marker_followers_bios'].iloc[287456])






#### Chunked data

In [ ]:
#Inspecting the chunked data
chunks = load_data_in_chunks(data_path, chunksize= 1000)
chunks.keys()



# Inspect the first chunk of each dataset
for name, chunk_generator in chunks.items():
    if name != 'readme':
        print(f"First chunk of {name}:")
        print(next(chunk_generator))
    else:
        print(f"Contents of {name}:")
        print(chunk_generator)

First chunk of bios:
      twitter_name          id    screen_name             name  \
0             Nike   415859364           Nike             Nike   
1         adidasFR    25487201       adidasFR    adidas France   
2          Lacoste    18481641        Lacoste          Lacoste   
3         Converse   183398746       Converse         Converse   
4          Moncler   187897893        Moncler          Moncler   
..             ...         ...            ...              ...   
232      Cdiscount    63142684      Cdiscount        Cdiscount   
233           Fnac     8806412           Fnac             Fnac   
234  rueducommerce    16933084  rueducommerce  Rue du Commerce   
235         vinted  2228641753         vinted           Vinted   
236   AmazonFrance  3823963875   AmazonFrance        Amazon.fr   

                                           description  \
0                 #BlackLivesMatter and #StopAsianHate   
1                         𝕭𝖎𝖊𝖓𝖛𝖊𝖓𝖚𝖊 𝖆𝖚 𝕮𝖑𝖚𝖇 𝕺𝖗𝖎𝖌𝖎𝖓𝖆𝖑𝖘.   
2    Moving 

In [ ]:
chunks.keys()

dict_keys(['bios', 'followers_bios', 'followers', 'friends', 'readme'])

/home/livtollanes/miniconda3/envs/10.1thesis/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (0,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


In [ ]:
last_chunk = None
last_chunk_number = None

# Iterate over all chunks in the key 'followe_bios"
for i, chunk in enumerate(chunks):
    last_chunk = chunk
    last_chunk_number = i

# Print the shape of the last chunk
print(f"Chunk {last_chunk_number} shape: {last_chunk.shape}")

Chunk 36081 shape: (646, 21)


- There are 36 082 chunks. 
- All but last have shape 1000 x 21 (so far)
- The last one has 646 x 21

There are 36 mill rows (36,081,646)

In [ ]:
# Get the 'followers_bios' chunk
followers_bios_chunk = chunks['followers_bios']

# Print the shape of the 'followers_bios' chunk
print(f"followers_bios shape: {followers_bios_chunk.shape}")

TypeError: 'TextFileReader' object is not subscriptable

In [ ]:
total_chunks = 36082
rows_per_chunk = 1000
rows_in_last_chunk = 646

total_rows = (total_chunks - 1) * rows_per_chunk + rows_in_last_chunk
print(f"Total rows: {total_rows}")

Total rows: 36081646


In [ ]:


def inspect_problematic_row(data_path, key, row_number):
    chunksize = 1000  # Adjust as needed
    file_path = data_path + '/' + key + '.csv'
    i = 0
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        chunk.reset_index(inplace=True)
        if row_number < (i + 1) * chunksize and row_number >= i * chunksize:
            if row_number % chunksize < len(chunk):
                return chunk.loc[row_number % chunksize], row_number
        i += 1

# Usage
key = 'markers_followers_bios_2023-05-19'
row_number = 287131
problematic_row, original_row_number = inspect_problematic_row(data_path, key, row_number)
print(f"Original row number: {original_row_number}")
print("Row data:")
print(problematic_row)

Original row number: 287131
Row data:
index                                                               287131
twitter_id                                             1614030263431974912
id                                                     1614030263431974912
screen_name                                                    PatriceVrai
name                                                       Patrice le vrai
description                                                            NaN
url                                                                    NaN
timestamp_utc                                                   1673649794
local_time                                             2023-01-13T22:43:14
location                                                               NaN
verified                                                                 0
protected                                                                0
tweets                                                        

In [ ]:
import pandas as pd

def inspect_problematic_row(data_path, key, row_number):
    chunksize = 10000  # Adjust as needed
    file_path = data_path + '/' + key + '.csv'
    i = 0
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        if row_number < (i + 1) * chunksize:
            return chunk.loc[row_number % chunksize]
        i += 1

# Usage
data_path = "/home/livtollanes/SocialMarkers"
key = 'marker_followers_bios'
row_number = 287131
problematic_row = inspect_problematic_row(data_path, key, row_number)
print(problematic_row)

### New approach, only load one data set

In [31]:
path_f_bios = "/home/livtollanes/SocialMarkers/markers_followers_bios_2023-05-19.csv"


df = dd.read_csv(path_f_bios, dtype='object')

print(type(df))

#inspect the first few rows of the ds dataframe
print(df.columns)


<class 'dask.dataframe.core.DataFrame'>
Index(['twitter_id', 'id', 'screen_name', 'name', 'description', 'url',
       'timestamp_utc', 'local_time', 'location', 'verified', 'protected',
       'tweets', 'followers', 'friends', 'likes', 'lists', 'image',
       'default_profile', 'default_profile_image', 'witheld_in_countries',
       'witheld_scope'],
      dtype='object')


In [32]:
subset = df[['twitter_id', 'id', 'screen_name', 'name', 'description', 'location']]
subset.head()

,twitter_id,id,screen_name,name,description,location
0,3342215494,3342215494,titisanogo8,Titi sanogo,Je crois en DIEU et à mon travail j'y arrivera...,"Ile-de-France, France"
1,3115495713,3115495713,AndreDeybach,DEYBACH André,NaN,NaN
2,244075010,244075010,matttownley1985,Matt Townley,"Hotelier, traveller, fan of all things hospita...","Manchester, England"
3,2986463442,2986463442,alex_guevara90,Alex,to all MI b**** what's up,NaN
4,1519279241858330625,1519279241858330625,PilotDad87,Matt,Pilot. dad. husband. Catholic. Recteq master. ...,"Greenville, NC"


In [33]:
#find the row length of the subset for column id
len(subset.id)


ParserError: Error tokenizing data. C error: EOF inside string starting at row 287131

In [57]:
# Determine the number of partitions in your data
npartitions = subset.npartitions
print(f"number of partitions: {npartitions}")

#dind number of rows and columns in selected partition
df_partition = subset.partitions[0].compute()

# Print the number of rows and columns
print(f"Number of rows: {df_partition.shape[0]}")
print(f"Number of columns: {df_partition.shape[1]}")

# I know the parsing error occurs in the first partition

# Load the first partition into memory as a pandas DataFrame
# df_partition = subset.get_partition(0).compute()

# # Get the rows around the parsing error
# selected_rows = df_partition.iloc[-55:-45]

# # Print the selected rows
# #print(selected_rows)
# df_partition.head()




number of partitions: 246
Number of rows: 287182
Number of columns: 6


In [58]:
#how many unique locations exist in the follower_bios dataframe?
# Convert the location column to lowercase and calculate the number of unique values
n_unique_locations = subset['location'].str.lower().nunique().compute()

print(f"Number of unique locations: {n_unique_locations}")

ParserError: Error tokenizing data. C error: EOF inside string starting at row 287334

### Data merging for french entries only

Total number of rows in followers_bios: 0
